In [4]:
from pypower.api import loadcase
import oct2py
import numpy as np
import os
from oct2py import Oct2Py
octave = Oct2Py()

In [6]:
os.getcwd()

'/home/exx/shourya/presolve/pglib-opf'

In [10]:
# get all cases in current directory
current_directory = os.getcwd()
all_files_and_directories = os.listdir(current_directory)
case_files = [os.path.abspath(f) for f in all_files_and_directories if f.endswith('.m') and os.path.isfile(os.path.join(current_directory, f))]
for cf in case_files:
    print(f"{os.path.basename(cf)}",flush=True)
    octave.eval(os.path.basename(cf))

pglib_opf_case60_c.m


Oct2PyError: Octave evaluation error:
error: structure has no member 'm'